In [63]:
import cv2
import os
import torch
import json
import numpy as np
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from mmcv.transforms import Compose
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def read_txt(txt_path):
    with open(txt_path) as f:
        data = f.readlines()
    data = [x.strip() for x in data]
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

In [116]:
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 1000
BATCH_SIZE = 64

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)
eval_data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)
calibration_images = read_txt('/teamspace/studios/this_studio/aimet/Examples/torch/quantization/calibration_image_ids.txt')
calibration_data_loader = DataLoader(calibration_images, batch_size=BATCH_SIZE)
DEVICE

Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth


The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


device(type='cuda', index=0)

In [27]:
# import inspect

# lines = inspect.getsource(dict(model.model.named_modules())['backbone.stem.1.bn'].__class__)
# print(lines)

In [65]:
from collections import OrderedDict
from copy import deepcopy

def replace_rtm_bn(model):
    m = deepcopy(model.model)

    def is_leaf(module): 
        return len(module._modules) == 0

    def replace_bn(m):

        if is_leaf(m):
            return 

        for _, child in m.named_children(): 
            
            if "bn" in child._modules.keys():
                bn = child._modules.get("bn")
                bn_params = deepcopy(bn._parameters)
                bn_buffers = deepcopy(bn._buffers)
                new_bn = torch.nn.BatchNorm2d(bn.num_features, eps=bn.eps, momentum=bn.momentum, affine=bn.affine, track_running_stats=bn.track_running_stats)
                new_bn._parameters["weight"].data = bn_params["weight"].data
                new_bn._parameters["bias"].data = bn_params["bias"].data
                new_bn._buffers["running_mean"].data = bn_buffers["running_mean"].data
                new_bn._buffers["running_var"].data = bn_buffers["running_var"].data
                new_bn._buffers["num_batches_tracked"].data = bn_buffers["num_batches_tracked"].data
                child._modules["bn"] = new_bn
                
            replace_bn(child)

    replace_bn(m)

    return m

In [29]:
from tqdm import tqdm
import torch

from mmdet.models.utils import samplelist_boxtype2tensor
from mmengine.registry import MODELS
from mmcv.transforms import Compose

test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

collate_preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True

preprocessor = MODELS.build(model.cfg.model.data_preprocessor)
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

loading annotations into memory...


Done (t=1.16s)
creating index...
index created!


In [30]:
def pass_calibration_data(model: torch.nn.Module, samples: int):
    data_loader = eval_data_loader
    batch_size = data_loader.batch_size
    model.eval()
    batch_ctr = 0
    with torch.no_grad():
        for image_path in tqdm(calibration_data_loader):
            image_path = [os.path.join(IMAGES_DIR, x) for x in image_path]
            pre_processed = collate_preprocessor(inputs=image_path, batch_size=batch_size)
            _, data = list(pre_processed)[0]
            data = preprocessor(data, False)
            
            preds = model(data['inputs'].to(DEVICE))  

            # batch_ctr += 1
            # if (batch_ctr * batch_size) > samples:
            #     break

In [15]:
dict(model.model.named_modules())['backbone.stage1.1.main_conv.bn'].weight

Parameter containing:
tensor([1.3939, 0.4195, 0.4683, 0.4950, 0.5379, 0.4248, 0.3259, 1.5808, 1.2741,
        0.5667, 0.2874, 0.1334, 0.3919, 0.5569, 0.6871, 0.5312, 0.8215, 0.9876,
        0.2597, 0.6685, 0.6328, 0.7639, 0.1999, 0.3412], device='cuda:0',
       requires_grad=True)

In [17]:
# from aimet_torch.model_preparer import prepare_model

# class CustomBatchNorm2d(torch.nn.Module):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.bn = torch.nn.BatchNorm2d(num_features = 12, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True)
    
#     def forward(self, x):
#         return self.bn(x)
    
# class CustomBatchNorm2d(torch.nn.modules.batchnorm._BatchNorm):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#     def _check_input_dim(self, input: torch.Tensor):
#         return

# class CustomModel(torch.nn.Module):
#     def __init__(self): 
#         super().__init__()

#         self.conv = torch.nn.Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
#         self.bn = CustomBatchNorm2d(num_features = 12, eps = 1e-05, momentum = 0.1, affine = True, track_running_stats = True)

#     def forward(self, x):
#         x = self.conv(x)
#         x = self.bn(x)
#         return x
    

# custom_model = CustomModel()

# model = prepare_model(custom_model)

# from aimet_torch.batch_norm_fold import fold_all_batch_norms

# fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

2024-09-04 13:06:36,143 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.module_add} 
2024-09-04 13:06:36,145 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.module_cat} 
2024-09-04 13:06:36,146 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.attention.module_mul} 
2024-09-04 13:06:36,148 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.blocks.0.module_add_1} 
2024-09-04 13:06:36,149 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.module_cat_1} 
2024-09-04 13:06:36,150 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.attention.module_mul_1} 
2024-09-04 13:06:36,151 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.blocks.0.module_add_2} 
2024-09-04 13:06:36,152 - ModelPrep

In [32]:
len(bn_pairs)

76

In [16]:
from aimet_torch.v2.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme, QuantizationDataType
from aimet_torch.model_preparer import prepare_model
from aimet_torch.batch_norm_fold import fold_all_batch_norms

def exclude_modules_from_quant(sim, modules_to_ignore):
    name_to_quant_wrapper_dict = {}
    for name, module in sim.model.named_modules():
        name_to_quant_wrapper_dict[name] = module

    quant_wrappers_to_ignore = []
    for name in modules_to_ignore:
        quant_wrapper = name_to_quant_wrapper_dict[name]
        quant_wrappers_to_ignore.append(quant_wrapper)

    sim.exclude_layers_from_quantization(quant_wrappers_to_ignore)

dummy_input = torch.rand(1, 3, 640, 640).to(DEVICE)# Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)

m = replace_rtm_bn(model)
m = prepare_model(m)
bn_pairs = fold_all_batch_norms(m, input_shapes=(1, 3, 640, 640))
print("Length of bn pairs: ", len(bn_pairs))

modules = dict(m.named_modules())

modules_to_change = ['backbone.stage2.1.blocks.0.conv2.depthwise_conv.bn', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn', 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn', 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn', 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn', 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn', 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn', 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn']
modules_to_change = {modules[x]: {"output_bw": 16, "param_bw": 16, "data_type": QuantizationDataType.float, "module_name": x} for x in modules_to_change}

print("dtype of model: ", list(dict(m.named_parameters()).values())[0].dtype)
quant_sim = QuantizationSimModel(model=m,
                                quant_scheme=QuantScheme.post_training_tf_enhanced,
                                default_param_bw=8,
                                default_output_bw=8,
                                config_file=None,
                                dummy_input=dummy_input,
                                modules_to_change=modules_to_change,
                                in_place=True)

### if load encodings
# quant_sim.load_encodings(encodings="/teamspace/studios/this_studio/aimet/Examples/torch/quantization/sim_model_excluded_modules/rtm_det_torch.encodings")
# quant_sim.load_encodings(encodings="/teamspace/studios/this_studio/aimet/Examples/torch/quantization/quant_scheme_W@tf / A@tf/rtm_det_torch.encodings")
# quant_sim.load_encodings(encodings=f"{BASE_PATH}/rtm_det_torch.encodings")

### else compute encodings
# quant_sim.compute_encodings(pass_calibration_data, 1000)

# modules_to_ignore = ['backbone.stage2.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_14', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_7', 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_21', 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_30', 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_37', 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_44', 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_51', 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn.module_batch_norm_58']
# exclude_modules_from_quant(quant_sim, modules_to_ignore)

2024-09-04 12:32:06,567 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.module_add} 
2024-09-04 12:32:06,568 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.module_cat} 
2024-09-04 12:32:06,570 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.attention.module_mul} 
2024-09-04 12:32:06,571 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.blocks.0.module_add_1} 
2024-09-04 12:32:06,572 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.module_cat_1} 
2024-09-04 12:32:06,573 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.attention.module_mul_1} 
2024-09-04 12:32:06,574 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.blocks.0.module_add_2} 
2024-09-04 12:32:06,575 - ModelPrep

2024-09-04 12:32:14,620 - BatchNormFolding - INFO - 0 BatchNorms' weights got converted
Length of bn pairs:  76
dtype of model:  torch.float32
2024-09-04 12:32:18,710 - Quant - INFO - Changed quantization params for this module: backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv
2024-09-04 12:32:18,781 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-09-04 12:32:18,833 - Quant - INFO - Unsupported op type Squeeze
2024-09-04 12:32:18,834 - Quant - INFO - Unsupported op type Mean
2024-09-04 12:32:18,858 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


In [18]:
print(str(quant_sim))

-------------------------
Quantized Model Report
-------------------------
GraphModule(
  (backbone): Module(
    (stem): Module(
      (0): Module(
        (conv): QuantizedConv2d(
          3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=[1], bitwidth=8, symmetric=True)
            (bias): None
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=[1], bitwidth=8, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=[1], bitwidth=8, symmetric=False)
          )
        )
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): QuantizedSigmoid(
            (param_quantizers): ModuleDict()
            (input_quantizers): ModuleList(
              (0): None
            )
            (output_quantizers): ModuleList(
              (0): QuantizeDequantize(shape=[1], bitw

In [19]:
### else compute encodings
quant_sim.compute_encodings(pass_calibration_data, 1000)

100%|██████████| 16/16 [05:22<00:00, 20.18s/it]


In [20]:
import torch
import os
import traceback
import shutil
dummy_input = torch.rand(1, 3, 640, 640)
output_dir = f"/teamspace/studios/this_studio/aimet/exported_models/bn_folded_fp16_encodings"
os.makedirs(output_dir, exist_ok=True)
quant_sim.export(path=output_dir,
            filename_prefix="rtm_det",
            dummy_input=dummy_input.cpu())

output_dir = f"/teamspace/studios/this_studio/aimet/exported_models/bn_folded_fp16_embdedded"
os.makedirs(output_dir, exist_ok=True)
try:
    quant_sim.export(path=output_dir,
                filename_prefix="rtm_det",
                dummy_input=dummy_input.cuda(),
                use_embedded_encodings=True,
                export_to_torchscript=False)
except:
    shutil.rmtree(output_dir, ignore_errors=True)
    traceback.print_exc()

2024-09-04 12:37:43,026 - Quant - WARNING - Exporting encodings to yaml will be deprecated in a future release. Ensure that your code can work with the exported files ending in ".encodings" which are saved using json format. For the time being, if yaml export is needed, set aimet_common.utils.SAVE_TO_YAML to True.


[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W s

2024-09-04 12:37:51,009 - Utils - INFO - successfully created onnx model with 274/279 node names updated
2024-09-04 12:37:51,568 - Quant - INFO - Layers excluded from quantization: []
2024-09-04 12:37:51,619 - Quant - WARNING - Exporting encodings to yaml will be deprecated in a future release. Ensure that your code can work with the exported files ending in ".encodings" which are saved using json format. For the time being, if yaml export is needed, set aimet_common.utils.SAVE_TO_YAML to True.


Traceback (most recent call last):
  File "/tmp/ipykernel_156794/714565966.py", line 15, in <module>
    quant_sim.export(path=output_dir,
  File "/usr/local/lib/python3.10/dist-packages/aimet_torch/v2/quantsim/quantsim.py", line 171, in export
    return super().export(path, filename_prefix, dummy_input, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/aimet_torch/quantsim.py", line 553, in export
    QuantizationSimModel.save_model_with_embedded_quantization_nodes(self.model, path, filename_prefix, dummy_input,
  File "/usr/local/lib/python3.10/dist-packages/aimet_torch/quantsim.py", line 2054, in save_model_with_embedded_quantization_nodes
    OnnxSaver._export_model_to_onnx(quant_sim_model, dummy_input, model_path, is_conditional, onnx_export_args) # pylint: disable=protected-access
  File "/usr/local/lib/python3.10/dist-packages/aimet_torch/onnx_utils.py", line 1510, in _export_model_to_onnx
    torch.onnx.export(model, dummy_input, temp_file, **kwargs)
  File "/us

In [67]:
model.model.eval()
m.eval()

GraphModule(
  (backbone): Module(
    (stem): Module(
      (0): Module(
        (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
      (1): Module(
        (conv): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
      (2): Module(
        (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
    )
    (stage1): Module(
      (0): Module(
        (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multipl

In [125]:
from aimet_torch.v2.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme, QuantizationDataType
from aimet_torch.model_preparer import prepare_model
from aimet_torch.batch_norm_fold import fold_all_batch_norms

m_replaced = replace_rtm_bn(model)
m = prepare_model(m_replaced)
bn_pairs = fold_all_batch_norms(m, input_shapes=(1, 3, 640, 640))
print(len(bn_pairs))
m_replaced.eval()
m.eval()

2024-09-04 13:25:41,785 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.module_add} 
2024-09-04 13:25:41,786 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.module_cat} 
2024-09-04 13:25:41,786 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.attention.module_mul} 
2024-09-04 13:25:41,787 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.blocks.0.module_add_1} 
2024-09-04 13:25:41,788 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.module_cat_1} 
2024-09-04 13:25:41,789 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.attention.module_mul_1} 
2024-09-04 13:25:41,790 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.blocks.0.module_add_2} 
2024-09-04 13:25:41,790 - ModelPrep

2024-09-04 13:25:47,374 - BatchNormFolding - INFO - 0 BatchNorms' weights got converted
76


GraphModule(
  (backbone): Module(
    (stem): Module(
      (0): Module(
        (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
      (1): Module(
        (conv): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
      (2): Module(
        (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
    )
    (stage1): Module(
      (0): Module(
        (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): Identity()
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multipl

In [126]:
import torch.nn as nn

conv_module = nn.Sequential(*list(dict(m_replaced.named_modules())['backbone.stem.0'].children()))[:-1]
conv_module.eval()
print(conv_module)

with torch.no_grad():
    out = conv_module.cuda()(dummy_input.cuda())

Sequential(
  (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [127]:
conv_module_bn = nn.Sequential(*list(dict(m.named_modules())['backbone.stem.0'].children()))[:-1]
conv_module_bn.eval()
print(conv_module_bn)

with torch.no_grad():
    bn_output = conv_module_bn.cuda()(dummy_input.cuda())

Sequential(
  (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (1): Identity()
)


In [128]:
# check if 2 matrices are equal
(out == bn_output).all()

tensor(False, device='cuda:0')

In [129]:
# calc diff between 2 matrices
(out - bn_output).abs()

tensor([[[[4.3141, 4.4292, 4.5622,  ..., 4.5342, 4.5249, 4.6823],
          [4.4157, 4.5130, 4.4154,  ..., 4.3527, 4.4362, 4.5275],
          [4.4246, 4.5103, 4.2153,  ..., 4.4685, 4.3702, 4.0950],
          ...,
          [4.3897, 4.2957, 4.2415,  ..., 4.4987, 4.3232, 4.4769],
          [4.1684, 4.2861, 4.4192,  ..., 4.4097, 4.4444, 4.5010],
          [4.5005, 4.3415, 4.3780,  ..., 4.1872, 4.4633, 4.2803]],

         [[2.6428, 2.6722, 2.7447,  ..., 2.6616, 2.6650, 2.6964],
          [2.6542, 2.6823, 2.6110,  ..., 2.6378, 2.6144, 2.6767],
          [2.6883, 2.6874, 2.5713,  ..., 2.6770, 2.6748, 2.5529],
          ...,
          [2.6759, 2.6374, 2.6226,  ..., 2.6757, 2.7020, 2.7490],
          [2.5737, 2.6223, 2.6866,  ..., 2.6789, 2.6208, 2.6766],
          [2.7340, 2.5968, 2.6172,  ..., 2.5728, 2.6419, 2.6026]],

         [[0.7581, 1.1461, 0.9555,  ..., 2.0014, 1.1080, 0.9551],
          [1.3004, 1.9084, 0.3630,  ..., 1.2525, 0.9352, 1.3551],
          [2.1100, 2.1703, 2.5564,  ..., 2